# DEDL - HDA Tutorial

This notebook demonstrates how to use the HDA (Harmonized Data Access) API by sending a few HTTP requests to the API, using Python code.

The detailed API and definition of each endpoint and parameters is available in the HDA Swagger UI at:

https://hda.data.destination-earth.eu/docs/

## Import the relevant modules
We start off by importing the relevant modules for HTTP requests and json handling, as well as writing a small pretty printing helper for viewing json responses in the notebook.

In [2]:
from typing import Union
import requests
import json
import urllib.parse
from requests.auth import HTTPBasicAuth

from IPython.display import JSON

def display_as_json(response: requests.Response) -> None:
    """Displays a HTTP request response as an interactive JSON in Jupyter Hub.
    
    Args:
        response (requests.Response): HTTP request response
    Returns:
        None
    """
    if not isinstance(response, requests.Response):
        raise TypeError(f"display_as_json expects a requests.Response parameter, got {type(response)}.")
    return JSON(json.loads(response.text))


## Define some constants for the API URLs
In this section, we define the relevant constants, holding the URL strings for the different endpoints.

In [4]:
# IDS
SERVICE_ID = "de-ecmwf-polytope-lumi"

# Use EUM Collection https://hda.central.data.destination-earth.eu/ui/dataset/EO.EUM.DAT.SENTINEL-3.OL_1_EFR___
COLLECTION_ID = "EO.EUM.DAT.SENTINEL-3.OL_1_EFR___"
COLLECTION_ID = "EO.ESA.DAT.SENTINEL-2.MSI.L1C"

ITEM_ID = "S2A_MSIL1C_20230910T050701_N0509_R019_T47VLH_20230910T074321.SAFE"

# Core API
HDA_API_URL = "https://hda.data.destination-earth.eu"
SERVICES_URL = f"{HDA_API_URL}/services"
SERVICE_BY_ID_URL = f"{SERVICES_URL}/{SERVICE_ID}"

# STAC API
## Core
STAC_API_URL = f"{HDA_API_URL}/stac"
CONFORMANCE_URL = f"{STAC_API_URL}/conformance"

## Item Search
SEARCH_URL = f"{STAC_API_URL}/search"
DOWNLOAD_URL = f"{STAC_API_URL}/download"

## Collections
COLLECTIONS_URL = f"{STAC_API_URL}/collections"
COLLECTION_BY_ID_URL = f"{COLLECTIONS_URL}/{COLLECTION_ID}"

## Items
COLLECTION_ITEMS_URL = f"{COLLECTIONS_URL}/{COLLECTION_ID}/items"
COLLECTION_ITEM_BY_ID_URL = f"{COLLECTIONS_URL}/{COLLECTION_ID}/items/{ITEM_ID}"

## HTTP Success
HTTP_SUCCESS_CODE = 200

In [5]:
# Generate an access token
# An access token is required to access and download items with the HDA API.
from getpass import getpass

username = ''
password = ''

response = requests.post(
    'https://identity.data.destination-earth.eu/auth/realms/dedl/protocol/openid-connect/token',
    data = {'grant_type': 'password','scope' : 'openid', 'client_id' : 'hda-public', 
            'username' : username, 'password' : password},
    headers = {"Content-Type" : "application/x-www-form-urlencoded"}
)

response.raise_for_status()

access_token = response.json()['access_token']

HTTPError: 401 Client Error: Unauthorized for url: https://identity.data.destination-earth.eu/auth/realms/dedl/protocol/openid-connect/token

## Core API

We can start off by requesting the HDA landing page, which provides links to the API definition (links `service-desc` and `service-doc`), the available services as well as the STAC API index.  

In [4]:
print(HDA_API_URL)
display_as_json(requests.get(HDA_API_URL))

https://hda.data.destination-earth.eu


<IPython.core.display.JSON object>

### Services
Requesting the `/services` endpoint will return the list of services available for users of the platform.
It fetches the services from the **services catalog** database.

In [6]:
print(SERVICES_URL)
display_as_json(requests.get(SERVICES_URL))

https://hda.data.destination-earth.eu/services


<IPython.core.display.JSON object>

The API can also describe a specific service, identified by its `serviceID` (e.g. **POLYTOPE**).

In [7]:
print(SERVICE_BY_ID_URL)
display_as_json(requests.get(SERVICE_BY_ID_URL))

https://hda.data.destination-earth.eu/services/de-ecmwf-polytope-lumi


<IPython.core.display.JSON object>

## STAC API
The HDA is plugged to a STAC API, component of the **EO Catalogue**.
### Core
The STAC API entry point is set to the `/stac` endpoint and provides the search capabilities provided by the DEDL STAC interface.

In [24]:
print(STAC_API_URL)
display_as_json(requests.get(STAC_API_URL))

https://hda.lumi.data.destination-earth.eu/stac


<IPython.core.display.JSON object>

The user can also have access to the list of all the conformance classes that the server conforms to by hitting the `/stac/conformance` endpoint.

In [7]:
print(CONFORMANCE_URL)
display_as_json(requests.get(CONFORMANCE_URL))

https://hda.central.data.destination-earth.eu/stac/conformance


<IPython.core.display.JSON object>

### Collections
The `/stac/collections` endpoint returns a `FeatureCollection` object, listing all STAC collections available to the user.

In [8]:
print(COLLECTIONS_URL)
display_as_json(requests.get(COLLECTIONS_URL))

https://hda.central.data.destination-earth.eu/stac/collections


<IPython.core.display.JSON object>

By providing a specific `collectionID` (e.g. **EO.ESA.DAT.SENTINEL-1.L1_GRD**), the user can get the metadata for a specific `Collection`.

In [9]:
print(COLLECTION_BY_ID_URL)
display_as_json(requests.get(COLLECTION_BY_ID_URL))

https://hda.central.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-2.MSI.L1C


<IPython.core.display.JSON object>

### Items
It is also possible to get the list of items available in a given `Collection` using Simple Search.

In [10]:
DATETIME = "?datetime=2023-09-09T00:00:00Z/2023-09-20T23:59:59Z"
print(COLLECTION_ITEMS_URL+DATETIME)
display_as_json(requests.get(COLLECTION_ITEMS_URL+DATETIME, headers={'Authorization': 'Bearer {}'.format(access_token)}))

https://hda.central.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-2.MSI.L1C/items?datetime=2023-09-09T00:00:00Z/2023-09-20T23:59:59Z


<IPython.core.display.JSON object>

### Item ID
To get the metadata specific to a given item (identified by its `itemID` in a collection, the user can request the `/stac/collections/{collectionID}/items/{itemID}`endpoint.

In [11]:
print(COLLECTION_ITEM_BY_ID_URL)
display_as_json(requests.get(COLLECTION_ITEM_BY_ID_URL, headers={'Authorization': 'Bearer {}'.format(access_token)}))

https://hda.central.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-2.MSI.L1C/items/S2A_MSIL1C_20230910T050701_N0509_R019_T47VLH_20230910T074321.SAFE


<IPython.core.display.JSON object>

### Item search
The STAC API also provides an item `/stac/search` endpoint, which is intended as a shorthand API for simple queries.
This endpoint allows users to efficiently search for items that match the specified input filters.

By default, the `/stac/search` endpoint will return the first 100 items found in all the collections available at the `/stac/collections` endpoint.
Filters can be added either via query parameters in a **GET** request or added to the JSON body of a **POST** request.

The user can also enhance the request by adding filters. The full detail for each available filter is available in the [API documentation](https://hda.data.destination-earth.eu/docs/).

The query parameters are added at the end of the URL as a *query string*: `?param1=val1&param2=val2&param3=val3`

In [12]:
SEARCH_QUERY_STRING = "?collections=EO.ESA.DAT.SENTINEL-1.L1_GRD&datetime=2023-09-09T00:00:00Z/2023-09-20T00:00:00Z"
print(SEARCH_URL + SEARCH_QUERY_STRING)
display_as_json(requests.get(SEARCH_URL + SEARCH_QUERY_STRING, headers={'Authorization': 'Bearer {}'.format(access_token)}))

https://hda.central.data.destination-earth.eu/stac/search?collections=EO.ESA.DAT.SENTINEL-1.L1_GRD&datetime=2023-09-09T00:00:00Z/2023-09-20T00:00:00Z


<IPython.core.display.JSON object>

The same filters can be added as the JSON body of a **POST** request.

In [13]:
BODY = {
    "collections": [
        "EO.ESA.DAT.SENTINEL-1.L1_GRD",
    ],
    "datetime" : "2023-09-09T00:00:00Z/2023-09-20T23:59:59Z",
    "bbox": [-11,35,
              50,72 ],
    "limit": 10,
}
response = requests.post(SEARCH_URL, json=BODY, headers={'Authorization': 'Bearer {}'.format(access_token)})
display_as_json(response)

<IPython.core.display.JSON object>

On top of the `/stac/search` capabilities, the user is able to download the items found with the search endpoint by sending a **POST** request to the `/stac/download` endpoint.

Under the hood, the API gets the list of items by sending the same JSON body as a **POST** request to the `/stac/search` endpoint before trying to download them, adding the item assets to a `.zip` archive.

For example, to download one item of the **EO.ESA.DAT.SENTINEL-1.L1_GRD** collection, you would add the following json body to the **POST** `/stac/download` request :
```json
{
    "collections": [
        "EO.ESA.DAT.SENTINEL-1.L1_GRD",
    ],
    "limit": 1,
}
```

In [14]:
result = json.loads(response.text)
downloadUrl = result['features'][0]['assets']['downloadLink']['href']
print(downloadUrl)

response = requests.get(downloadUrl,stream=True,headers={'Authorization': 'Bearer {}'.format(access_token), 'Accept-Encoding': None})
    
# If the request was successful, download the file
if (response.status_code == HTTP_SUCCESS_CODE):
        print("Downloading dataset...")
        product_id = 'Item'
        filename = product_id + ".zip"
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    f.flush()
        print("The dataset has been downloaded to: {}".format(filename))
else: print("Request Unsuccessful! Error-Code: {}".format(response.status_code))

https://hda.central.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-1.L1_GRD/items/S1A_EW_GRDM_1SDH_20230909T031622_20230909T031722_050245_060C53_F1E9/download
Request Unsuccessful! Error-Code: 403


# Conclusion
This small tutorial presented the HDA API, illustrated with some pieces of Python code showing how to send HTTP requests to the different endpoints, as well as the use of a few filtering capabilities.

Again, more detail on each endpoint can be found in the [API documentation](https://hda.data.destination-earth.eu/docs/).

# FAQ

### My request is failing due to unsupported provider
A: This is totally normal and expected as your provider is not a valid host provider.

Valid Host Providers can be found in the collection metadata with providers having **host** as a listed role.

e.g 

```json
"roles": [

    "processing",

    "host"

]
```

### My `/search` or `/items` request return 0 items

Multiple reasons can lead to no items returned in the response

#### You are not authenticated

You did not provide an access token in the request. An access token is required to search and access items in datasets.

#### No parameters set in the request or too wide parameters

The request parameters are not precise enough for the HDA API to give you a response. Narrow down your request by giving more specific parameters.

e.g You can use a short datetime range like **datetime=2023-09-09T00:00:00Z/2023-09-21T00.00.00Z**

#### There is no item matching the request parameters

Modify the parameters to find items.


### Expected products within the specified datetime range from my `/search` or `/items` requests are missing from the results

As of now the search returns results rounded up to nearest milisecond. Please take this into account on your datetime queries.

e.g Should you want to access an item with datetime in its metadata `2023-01-01T12:22:33:555`. This item may have in reality the datetime `2023-01-01T12:22:33:55487655`. Hence, your search filter should be `2023-01-01T12:22:33:554/2023-01-01T12:22:33:556`.

